In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import skimage.io
import skimage.morphology

import os.path

import sys

sys.path.append('/home/jr0th/github/segmentation/code/')
import helper.metrics

import numpy as np

In [ ]:
# problem-specific paths
dir_root = '/home/jr0th/github/segmentation/experiments/CP/'

tag = 'CP'
in_dir_segm = '/home/jr0th/github/segmentation/data/BBBC022/raw/raw_cp_segmentations/'
out_dir_segm = dir_root + "segm_" + tag + '/'

path_result_txt = dir_root + 'results_table_' + tag + '.txt'

In [ ]:
# constant paths
in_dir_gt_segm = dir_root + '../GT_segmentations/'

out_dir_IoU = dir_root + 'IoU/'
path_files_test = '/home/jr0th/github/segmentation/data/BBBC022/test.txt'

In [ ]:
with open(path_files_test) as f:
    test_files = f.read().splitlines()

In [ ]:
# buffer for results
results = np.empty(shape = (0, 4))

# loop over all images
for filename in test_files:
    
    filename_wo_ext = os.path.splitext(filename)[0]
    
    # get model label
    mo_label = skimage.io.imread(in_dir_segm + filename_wo_ext + '.tiff')
    mo_label = skimage.morphology.label(mo_label)
    
    # get GT label
    gt_label = skimage.io.imread(in_dir_gt_segm + filename)
    
    # save CP label
    skimage.io.imsave(out_dir_segm + filename, mo_label)
    
    nb_gt_cells = np.max(gt_label)
    
    [nb_overdetection, nb_underdetection, mean_IoU, IoUs] = helper.metrics.compare_two_labels(mo_label, gt_label, True)
    
    np.save(out_dir_IoU + filename_wo_ext, IoUs)
    
    result = np.hstack((nb_overdetection, nb_underdetection, mean_IoU, nb_gt_cells))
    results = np.vstack((results, result))

np.savetxt(path_result_txt, results)